In [ ]:
# Unsloth설치
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from google.colab import drive

#구글 드라이브를 마운트합니다.
drive.mount('/content/drive')

output_dir = '/content/drive/MyDrive/outputs'

Mounted at /content/drive


In [ ]:
#파인튜닝 모델 로드
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    #위 모델에서 4bit모델이 T4에서 훈련이 가능하므로 호출
    model_name = fourbit_models[7],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from huggingface_hub import login

login("")

#명령어 구조
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}

### Response:
{}
"""

#End Of Sentence토큰의 끝을 알려주는 토큰 FILE의 EOF같은 것
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
#데이터셋 불러오기
dataset = load_dataset("MarkrAI/KOpen-HQ-Hermes-2.5-60K", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Generating train split:   0%|          | 0/60061 [00:00<?, ? examples/s]

Map:   0%|          | 0/60061 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

#파인튜닝을 위한 트레이너 셋팅
trainer = SFTTrainer(
    #타겟 모델
    model = model,
    #변환용 토크나이저
    tokenizer = tokenizer,
    #데이터 셋
    train_dataset = dataset,
    #데이터셋에서 하나의 레코드의 딕셔너리의 필드네임
    dataset_text_field = "text",
    #최대 토큰수
    max_seq_length = max_seq_length,
    #전처리 프로세스 수
    dataset_num_proc = 2,
    # 동시 처리용 - 현재 쓸모 없음
    packing = False,
    # 트레이닝 인자
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        #(learning_tate / warmup_steps) * step = 현재 스탭의 학습률(초기 학습률을 낮게 설정.)
        warmup_steps = 5,
        # 데이터셋을 몇번 학습할건지 epoch
        num_train_epochs = 1,
        #최대 몇 스탭까지 공부할 것인가?
        #max_steps = 150,
        #학습률
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        #몇 스탭마다 기록할지.
        logging_steps = 1,
        #최적화
        optim = "adamw_8bit",
        weight_decay = 0.01,
        #학습방법 현 선형적
        lr_scheduler_type = "linear",
        #랜덤성의 seed
        seed = 3407,
        #저장 위치
        output_dir = output_dir,
    ),
)

Map (num_proc=2):   0%|          | 0/60061 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# cuda 장치 상태 획득
gpu_stats = torch.cuda.get_device_properties(0)
#사용중인 메모리 획득
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
#GPU 최대 메모리
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.52 GB of memory reserved.


In [ ]:
#파인튜닝 시작
import os

#학습 체크포인트 폴더 유무를 체크합니다.
if not os.path.exists(output_dir):
    trainer_stats = trainer.train(resume_from_checkpoint = True)
else:
    trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 60,061 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.182300
2,1.259800
3,1.211200
4,1.185800
5,1.022500
6,1.004200
7,1.088000
8,1.046400
9,1.125100
10,1.015800


In [ ]:
#alpaca_prompt = 학습시 만들어둔 Instruct 구조
#한번에 입력
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "빠르게 변화하는 오늘날의 글로벌 시장에서 공급망 관리와 물류는 기업의 성공에 필수적인 요소가 되었습니다. 기업들은 효율성과 지속 가능성을 극대화하기 위해 이러한 프로세스를 최적화할 수 있는 방법을 끊임없이 모색하고 있습니다. 양자 컴퓨팅은 속도, 정확성, 확장성 향상을 통해 공급망 관리에 혁신을 가져올 수 있는 잠재력을 가진 유망한 기술 중 하나입니다. 하지만 공급망 관리의 복잡성과 기존 시스템 및 프로세스와의 원활한 통합이 필요하다는 점에서 양자 컴퓨팅을 공급망 관리에 구체적으로 어떻게 적용할 수 있을까요? 이 주제를 자세히 살펴보고 양자 컴퓨팅이 공급망 관리 환경을 어떻게 변화시킬 수 있는지 알아보세요.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n빠르게 변화하는 오늘날의 글로벌 시장에서 공급망 관리와 물류는 기업의 성공에 필수적인 요소가 되었습니다. 기업들은 효율성과 지속 가능성을 극대화하기 위해 이러한 프로세스를 최적화할 수 있는 방법을 끊임없이 모색하고 있습니다. 양자 컴퓨팅은 속도, 정확성, 확장성 향상을 통해 공급망 관리에 혁신을 가져올 수 있는 잠재력을 가진 유망한 기술 중 하나입니다. 하지만 공급망 관리의 복잡성과 기존 시스템 및 프로세스와의 원활한 통합이 필요하다는 점에서 양자 컴퓨팅을 공급망 관리에 구체적으로 어떻게 적용할 수 있을까요? 이 주제를 자세히 살펴보고 양자 컴퓨팅이 공급망 관리 환경을 어떻게 변화시킬 수 있는지 알아보세요.\n### Input:\n\n\n### Response:\n\n양자 컴퓨팅은 공급망 관리에 혁신을 가져올 수 있는 잠재력이 있습니다. 양자 컴퓨팅은 빠른 속도, 정확성, 확장성 향상을 통해 공급망 관리에 혁신을 가져올 수 있는 유망한 기술 중 하나입니다. 하지만 공급망 관리의 복잡성과 기존 시스템 및 프로세스와의 원활한 통합이 필요하다는 점에서 양자 컴퓨팅을 공급망 관리에 구체적으로 어떻게 적용할 수 있을까요? 이 주제를 자세히 살펴보고 양자 컴퓨팅이 공급망 관리 환경을 어떻게 변화시킬 수 있는지 알아보세요.\n\n양자 컴퓨팅은 공급망 관리에 혁신을 가져올 수 있는 잠재력이 있습니다. 양자 컴퓨팅은 빠른 속도, 정확성, 확장성 향상을 통해 공급망 관리에 혁신을 가져올 수 있는 유망한 기술 중 하나입니다. 하지만 공급망 관리의 복잡성과 기존 시스템 및 프로세스와의 원활한 통합이 필요하다는 점에서 양자 컴퓨팅을 공급망 관리에']

In [ ]:
#alpaca_prompt = 학습시 만들어둔 Instruct 구조
#스트림 입력처리
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "다음은 일부 데이터입니다: 미국, 익명, 미국인; 미국, 수도, 워싱턴 D.C.; Atlas II, 국가원산지, 미국; 미국, 민족그룹, 미국의 아메리카 원주민; 미국, 리더직함, 미국 대통령. 이 데이터에 대한 자세한 설명을 생성합니다. 문장:.", # instruction
        "귀하는 항상 설명을 제공하는 도움이 되는 조수입니다. 5살짜리 아이에게 대답한다고 생각하세요.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
다음은 일부 데이터입니다: 미국, 익명, 미국인; 미국, 수도, 워싱턴 D.C.; Atlas II, 국가원산지, 미국; 미국, 민족그룹, 미국의 아메리카 원주민; 미국, 리더직함, 미국 대통령. 이 데이터에 대한 자세한 설명을 생성합니다. 문장:.
### Input:
귀하는 항상 설명을 제공하는 도움이 되는 조수입니다. 5살짜리 아이에게 대답한다고 생각하세요.

### Response:

미국은 익명이 아닌 미국인이 거주하는 곳입니다. 미국은 워싱턴 D.C.라는 수도도 있습니다. 미국은 아메리카 원주민이 아닌 미국인이 거주하는 곳입니다. 미국은 미국 대통령이 있습니다.
</s>


In [ ]:
# import shutil
# import os

# quantization_methods = ["f16", "q8_0", "q4_k_m"]

# for method in quantization_methods:
#     # Save model to a temporary directory
#     output_dir = f"model_{method}"

#     # 양자화된 모델을 저장
#     model.save_pretrained_gguf(
#         output_dir,
#         tokenizer,
#         quantization_method=method
#     )

#     # Hugging Face Hub에 모델 업로드
#     model.push_to_hub_gguf(
#         "repository 주소",
#         tokenizer,
#         quantization_method=method,
#         token="토큰",
#     )

#     print(f"Uploaded {method} version to the same repository.")

#     # 로컬에 저장된 모델 데이터 삭제
#     shutil.rmtree(output_dir)
#     print(f"Deleted local files for {method} version.")
